In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import imghdr
from moviepy.editor import VideoFileClip
from IPython.display import HTML


from _lib_.undistort import camera

%matplotlib inline

In [ ]:
project_video = "./project_video.mp4"
clb_path = "./camera_cal"
test_path = "./test_images"
out_path = "./output_images"

## Camera Calibration and Fix Distortion

In [ ]:
samples = 5
chessboard_fs = [os.path.join(clb_path, f) for f in sorted(os.listdir(clb_path))[:samples]]
print(chessboard_fs)
chessboards = [cv2.imread(f) for f in chessboard_fs]

In [ ]:
cam = camera(clb_path, 9, 6)
undst_chessboards = [cam.cal_undist(cb) for cb in chessboards]

In [ ]:
print("distort and undisort images")
plt.rcParams['figure.figsize'] = (samples, 8)
fig, axes = plt.subplots(samples, 2)
for i in range(samples):
    axes[i][0].imshow(chessboards[i])
    axes[i][1].imshow(undst_chessboards[i])
    axes[i][0].axis('off')
    axes[i][1].axis('off')
    f = os.path.join(out_path, clb_path[2:], ("undist_calibration%02d.jpg"%(i+1)))
    cv2.imwrite(f, undst_chessboards[i])
plt.show()

In [ ]:
corrected_video = 'corrected.mp4'
clip = VideoFileClip(project_video)
video = clip.fl_image(cam.cal_undist)
%time video.write_videofile(corrected_video, audio = False)

In [ ]:
HTML("""
<video width = "720" height = "405" controls>
<source src="{0}">
</video>
""".format(corrected_video))

In [ ]:
frame = clip.get_frame(3.78)
frame_corrected = video.get_frame(3.78)

plt.rcParams['figure.figsize'] = 20, 5
plt.subplot(121)
plt.imshow(frame)
plt.subplot(122)
plt.imshow(frame_corrected)

In [ ]:
test_imgfs = os.listdir(test_path)
n_test = len(test_imgfs)
test_out = os.path.join(out_path, "undist_test_images")
for imgf in test_imgfs:
    img = cv2.imread(os.path.join(test_path, imgf))
    corrected_img = cam.cal_undist(img)
    cv2.imwrite(os.path.join(test_out, ("undist_%s"%imgf)), corrected_img)

## Gradient Selection And Color Space 

In [ ]:
fig, axes = plt.subplots(n_test, 3, figsize = (15, n_test * 3))
fig.tight_layout()
for i, imgf in enumerate(test_imgfs):
    img = cv2.imread(os.path.join(test_path, imgf))
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    axes[i][0].imshow(hls[:,:,0], cmap = 'gray')
    axes[i][1].imshow(hls[:,:,1], cmap = 'gray')
    axes[i][2].imshow(hls[:,:,2], cmap = 'gray')
    for j in range(3):
        axes[i][j].axis('off')

plt.show()

In [ ]:
def select_color(im, l_thresh, s_thresh):
    img = np.copy(im)
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS).astype(np.float)
    h = hls[:,:,0]
    s = hls[:,:,2]
    mask = np.zeros_like(img[:,:,0])
    mask[(s >= s_thresh[0]) & (s <= s_thresh[1]) &
         (h >= l_thresh[0]) & (h <= l_thresh[1])] = 1
    return mask.astype(np.float32)

fig, axes = plt.subplots(n_test, 2, figsize = (10, n_test * 3))
fig.tight_layout()
plt.rcParams['figure.figsize'] = (2*n_test, 16)
for i, imgf in enumerate(test_imgfs):
    img = cv2.imread(os.path.join(test_path, imgf))
    axes[i][0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axes[i][1].imshow(
        select_color(img, l_thresh = (0, 100), s_thresh = (90, 255)),
        cmap = 'gray')
    axes[i][0].axis('off')
    axes[i][1].axis('off')

In [ ]:
def abs_sobel_thresh(img, sobel_kernel = 3, orient = 'x', thresh = (0, 255)):
    ch = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    x, y = int(orient is 'x'), int(orient is 'y')
    assert x + y == 1, "orientation should be 'x' or 'y'"
    abs_sobel = np.absolute(cv2.Sobel(ch, cv2.CV_64F, x, y, ksize = sobel_kernel))
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    mask = np.zeros_like(scaled_sobel)
    mask[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return mask.astype(np.float32)

In [ ]:
def mag_sobel_thresh(img, sobel_kernel = 3, thresh = (0, 255)):
    ch = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sobelx = cv2.Sobel(ch, cv2.CV_64F, 1, 0, ksize = sobel_kernel)
    sobely = cv2.Sobel(ch, cv2.CV_64F, 0, 1, ksize = sobel_kernel)
    sobel_mag = np.sqrt(sobelx**2 + sobely**2)
    mask = np.zeros_like(sobel_mag)
    mask[(sobel_mag >= thresh[0]) & (sobel_mag <= thresh[1])] = 1    
    return mask.astype(np.float32)

In [ ]:
def dir_sobel_thresh(img, sobel_kernel = 3, thresh = (0, np.pi/2)):
    ch = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sobelx = np.absolute(cv2.Sobel(ch, cv2.CV_64F, 1, 0, ksize = sobel_kernel))
    sobely = np.absolute(cv2.Sobel(ch, cv2.CV_64F, 0, 1, ksize = sobel_kernel))
    sobel_dir = np.arctan2(sobely, sobelx)
    mask = np.zeros_like(sobel_dir)
    mask[(sobel_dir >= thresh[0]) & (sobel_dir <= thresh[1])] = 1    
    return mask.astype(np.float32)

In [ ]:
fig, axes = plt.subplots(n_test, 2, figsize = (10, n_test * 3))
fig.tight_layout()
for i, imgf in enumerate(test_imgfs):
    img = cv2.imread(os.path.join(test_path, imgf))
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS).astype(np.float)
    s = hls[:,:,2]
    axes[i][0].imshow(s,cmap = 'gray')
    axes[i][1].imshow(abs_sobel_thresh(img, sobel_kernel = 3, thresh = (25, 255)),
        cmap = 'gray')
    axes[i][0].axis('off')
    axes[i][1].axis('off')

In [ ]:
fig, axes = plt.subplots(n_test, 2, figsize = (10, n_test * 3))
fig.tight_layout()
for i, imgf in enumerate(test_imgfs):
    img = cv2.imread(os.path.join(test_path, imgf))
    axes[i][0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    mask_gradient = abs_sobel_thresh(img, sobel_kernel = 3, thresh = (25, 255))
    mask_color = select_color(img, l_thresh = (0, 100), s_thresh = (90, 255))
    mask = np.dstack((np.zeros_like(img[:,:,0]), mask_gradient, mask_color))
    axes[i][1].imshow(mask)
    for j in range(2):
        axes[i][j].axis('off')

In [ ]:
standard_img = cv2.imread(os.path.join(test_path, test_imgfs[3]))
standard_img = cv2.cvtColor(standard_img, cv2.COLOR_BGR2RGB)
plt.rcParams['figure.figsize'] = (20, 40)

src = np.float32([[225, 710], [1095, 710], [545, 485], [745, 485]])

green = (0, 255, 0)
check = np.copy(standard_img)
for p in src:
    cv2.circle(check, tuple(p), 8, green, -5)

plt.subplot(121)
plt.imshow(standard_img)
plt.subplot(122)
plt.imshow(check)
plt.show()

In [ ]:
dst = np.float32([[225, 710], [1095, 710], [225, 460], [1095, 460]])
M = cv2.getPerspectiveTransform(src, dst)
h, w, _ = standard_img.shape
warped = cv2.warpPerspective(standard_img, M, (w, h), flags = cv2.INTER_LINEAR)
plt.imshow(warped)
plt.show()

In [ ]:
fig, axes = plt.subplots(n_test, 2, figsize = (10, n_test * 3))
fig.tight_layout()
for i, imgf in enumerate(test_imgfs):
    img = cv2.imread(os.path.join(test_path, imgf))
    axes[i][0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    warped = cv2.warpPerspective(img, M, (w, h), flags = cv2.INTER_LINEAR)
    mask_gradient = mag_sobel_thresh(warped, sobel_kernel = 15, thresh = (30, 255))
    mask_color = select_color(warped, l_thresh = (0, 90), s_thresh = (90, 255))
    mask = np.uint8(mask_gradient) + np.uint8(mask_color)
    #mask = np.dstack((np.zeros_like(img[:,:,0]), mask_gradient, mask_color))
    #warped = cv2.warpPerspective(mask, M, (w, h), flags = cv2.INTER_LINEAR)
    axes[i][1].imshow(mask, cmap = 'gray')
    for j in range(2):
        axes[i][j].axis('off')

In [ ]:
def dev_pipeline(img):
    undist = cam.cal_undist(img)
    warped = cv2.warpPerspective(undist, M, (w, h), flags = cv2.INTER_LINEAR)
    mask_gradient = abs_sobel_thresh(warped, sobel_kernel = 3, thresh = (25, 255))
    mask_color = select_color(warped, l_thresh = (0, 100), s_thresh = (90, 255))
    mask = mask_gradient + mask_color
    return np.tile(mask.reshape([h, w, 1]), (1, 1, 3))

In [ ]:
frame = clip.get_frame(3.78)
warped = dev_pipeline(frame)

plt.rcParams['figure.figsize'] = 20, 5
plt.subplot(121)
plt.imshow(frame)
plt.subplot(122)
plt.imshow(warped)

In [ ]:
lane = 'lane.mp4'
clip_lane = VideoFileClip(project_video)
video = clip_lane.fl_image(dev_pipeline)
%time video.write_videofile(lane, audio = False)

In [ ]:
HTML("""
<video width = "720" height = "405" controls>
<source src="{0}">
</video>
""".format(lane))